In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import dlib
import uuid
from tensorflow.keras.models import load_model
import os
import csv
from datetime import datetime

# Load emotion detection model
model = load_model(r"C:\Users\Ankita Borkar\OneDrive\Desktop\Emotion Detetction\emotion_model.h5")
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# Globals for tracking
trackers = {}
face_emotion_history = {}

# Emotion prediction
def predict_emotion(face_img):
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (48, 48))
    normalized = resized / 255.0
    reshaped = np.reshape(normalized, (1, 48, 48, 1))
    result = model.predict(reshaped, verbose=0)
    label_idx = int(np.argmax(result))
    confidence = float(np.max(result)) * 100
    return emotion_labels[label_idx], confidence

# Log to CSV
def log_emotion(frame_id, face_id, emotion, confidence):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open("emotion_log.csv", mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([frame_id, timestamp, face_id, emotion, f"{confidence:.2f}"])

# IoU for tracker matching
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea + 1e-5)

# For video & webcam
def detect_and_track_faces(frame, frame_id):
    global trackers, face_emotion_history
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detections = face_detection.process(rgb_frame)
    updated_trackers = {}

    # Update trackers
    for face_id, tracker in trackers.items():
        tracker.update(frame)
        pos = tracker.get_position()
        x, y, w, h = int(pos.left()), int(pos.top()), int(pos.width()), int(pos.height())
        x2, y2 = x + w, y + h

        face_roi = frame[y:y2, x:x2]
        if face_roi.size > 0:
            emotion, confidence = predict_emotion(face_roi)
            face_emotion_history[face_id] = (emotion, confidence)
            log_emotion(frame_id, face_id, emotion, confidence)
            label = f"ID: {face_id[:4]} | {emotion} ({confidence:.1f}%)"
            cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        updated_trackers[face_id] = tracker

    # New detections
    if detections and detections.detections:
        ih, iw, _ = frame.shape
        for det in detections.detections:
            bbox = det.location_data.relative_bounding_box
            x, y, w, h = int(bbox.xmin * iw), int(bbox.ymin * ih), int(bbox.width * iw), int(bbox.height * ih)
            new_box = (x, y, w, h)

            matched = False
            for tracker in updated_trackers.values():
                pos = tracker.get_position()
                existing_box = (int(pos.left()), int(pos.top()), int(pos.width()), int(pos.height()))
                if iou(new_box, existing_box) > 0.4:
                    matched = True
                    break

            if not matched:
                new_tracker = dlib.correlation_tracker()
                rect = dlib.rectangle(x, y, x + w, y + h)
                new_tracker.start_track(frame, rect)
                new_face_id = str(uuid.uuid4())
                updated_trackers[new_face_id] = new_tracker
                face_emotion_history[new_face_id] = ("Unknown", 0.0)

    trackers = updated_trackers
    return frame

# Fixed image processor
def process_image():
    path = input("Enter full image path: ").strip().strip('"')
    if not os.path.exists(path):
        print("[ERROR] Image not found!")
        return

    image = cv2.imread(path)
    if image is None:
        print("[ERROR] Could not open image.")
        return

    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    detections = face_detection.process(rgb_image)

    if detections and detections.detections:
        ih, iw, _ = image.shape
        for det in detections.detections:
            bbox = det.location_data.relative_bounding_box
            x, y, w, h = int(bbox.xmin * iw), int(bbox.ymin * ih), int(bbox.width * iw), int(bbox.height * ih)
            x, y = max(0, x), max(0, y)
            x2, y2 = min(iw, x + w), min(ih, y + h)

            face_roi = image[y:y2, x:x2]
            if face_roi.size > 0:
                emotion, confidence = predict_emotion(face_roi)
                label = f"{emotion} ({confidence:.1f}%)"
                cv2.rectangle(image, (x, y), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    else:
        print("[INFO] No face detected in image.")

    cv2.imshow("Emotion Detection - Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Video
def process_video():
    path = input("Enter full video path: ").strip().strip('"')
    if not os.path.exists(path):
        print("[ERROR] Video not found!")
        return

    cap = cv2.VideoCapture(path)
    frame_id = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = detect_and_track_faces(frame, frame_id)
        cv2.imshow("Emotion Detection - Video", frame)
        frame_id += 1

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Webcam
def process_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("[ERROR] Could not access webcam.")
        return

    frame_id = 0
    print("[INFO] Press 'q' to quit webcam.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = detect_and_track_faces(frame, frame_id)
        cv2.imshow("Emotion Detection - Webcam", frame)
        frame_id += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Menu
def main():
    print("\nEmotion Detection System")
    print("1. Process Image")
    print("2. Process Video")
    print("3. Process Webcam")
    choice = input("Choose an option (1/2/3): ").strip()

    if choice == '1':
        process_image()
    elif choice == '2':
        process_video()
    elif choice == '3':
        process_webcam()
    else:
        print("Invalid choice.")

# Create CSV if needed
if not os.path.exists("emotion_log.csv"):
    with open("emotion_log.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Frame ID", "Timestamp", "Face ID", "Emotion", "Confidence"])

# Run
if __name__ == "__main__":
    main()



Emotion Detection System
1. Process Image
2. Process Video
3. Process Webcam


Choose an option (1/2/3):  1
Enter full image path:  C:\Users\Ankita Borkar\Downloads\obama.jpg
